In [1]:
%pip install -r requirements.txt

   ---------------------------------------- 0.0/338.0 kB ? eta -:--:--
   --- ----------------------------------- 30.7/338.0 kB 660.6 kB/s eta 0:00:01
   ---- ---------------------------------- 41.0/338.0 kB 393.8 kB/s eta 0:00:01
   ------- ------------------------------- 61.4/338.0 kB 469.7 kB/s eta 0:00:01
   ------- ------------------------------- 61.4/338.0 kB 469.7 kB/s eta 0:00:01
   ------- ------------------------------- 61.4/338.0 kB 469.7 kB/s eta 0:00:01
   ---------- ---------------------------- 92.2/338.0 kB 350.1 kB/s eta 0:00:01
   ---------- ---------------------------- 92.2/338.0 kB 350.1 kB/s eta 0:00:01
   ---------------- --------------------- 143.4/338.0 kB 355.0 kB/s eta 0:00:01
   ------------------ ------------------- 163.8/338.0 kB 364.8 kB/s eta 0:00:01
   ------------------ ------------------- 163.8/338.0 kB 364.8 kB/s eta 0:00:01
   ------------------- ------------------ 174.1/338.0 kB 327.7 kB/s eta 0:00:01
   --------------------- ---------------- 194.6/3


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\tbhro\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


TypeError: Object of type Timestamp is not JSON serializable

TypeError: Object of type Timestamp is not JSON serializable

In [6]:
import geopandas as gpd
import pandas as pd
import folium

# Load data
issues_df = pd.read_csv('../data/challenge_2/complete_issues_data.csv')
states = gpd.read_file(
    "../vg5000_12-31.gk3.shape.ebenen/vg5000_ebenen_1231/VG5000_KRS.shp"
).to_crs("EPSG:4326")

# Count issues per state
issues_per_state = (
    issues_df.groupby('state')
    .size()
    .reset_index(name='issue_count')
)

# Merge
states_with_data = states.merge(
    issues_per_state,
    left_on='GEN',
    right_on='state',
    how='left'
)

# **Drop any datetime columns** and keep only GEN, issue_count, geometry
datetime_cols = states_with_data.select_dtypes(['datetime64[ns]']).columns
states_for_map = states_with_data.drop(columns=datetime_cols)[['GEN','issue_count','geometry']]

# Fill NaNs so tooltip shows 0 instead of “nan”
states_for_map['issue_count'] = states_for_map['issue_count'].fillna(0).astype(int)

# Build map
m = folium.Map(location=[51.0, 10.0], zoom_start=6)

folium.Choropleth(
    geo_data=states_for_map.to_json(),
    name='Issues by State',
    data=states_for_map,
    columns=['GEN', 'issue_count'],
    key_on='feature.properties.GEN',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Issues'
).add_to(m)

folium.features.GeoJson(
    states_for_map,
    name='State Info',
    tooltip=folium.features.GeoJsonTooltip(
        fields=['GEN', 'issue_count'],
        aliases=['State:', 'Issues:'],
        localize=True
    )
).add_to(m)

m.save('germany_issues_choropleth.html')
